In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from src.features import build_features

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION = [f"Poblacion {i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

In [3]:
ruta_base_de_datos = (
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
)

# Lee la base de datos
df = pd.read_excel(ruta_base_de_datos).iloc[:-2]

# Renombra columna de hombres o mujeres
df = df.rename(columns={"Sexo\n1=Hombre\n2=Mujer": "hombre_mujer"})

# Indica si es adulto o infantil
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO

In [4]:
ESTRATOS_A_CALCULAR = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR["SSMO"] = df_ssmo

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

In [5]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios

}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.

In [19]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR, query_strings, COLUMNAS_POBLACION
)
# Une todos los resultados en un unico DataFrame
poblaciones_estratos_calculados = pd.concat(poblaciones_estratos_calculados)

# Guarda resultados
poblaciones_estratos_calculados.to_excel("../data/interim/poblacion_INE_estratos.xlsx")

---

In [7]:
RUTA_PLANILLA_INCIDENCIAS = "../data/processed/proyeccion_problemas_de_salud.xlsx"
COLUMNAS_A_UTILIZAR = ["Diagnostico", "Incidencia (Cada 100.000)", "Edad Incidencia"]

# Lee planilla de incidencias
incidencias = pd.read_excel(
    RUTA_PLANILLA_INCIDENCIAS, sheet_name="incidencias_prevalencias", usecols=COLUMNAS_A_UTILIZAR
)

# Obtiene la fraccion de la incidencia, para multiplicar con la poblacion directamente
incidencias["rate_incidencia"] = incidencias["Incidencia (Cada 100.000)"] / 100000

In [11]:
for index, fila in incidencias.iterrows():
    nombre_diagnostico = fila["Diagnostico"]
    codigo_diagnostico = nombre_diagnostico[:4]
    incidencia = fila["Incidencia (Cada 100.000)"]
    rate_incidencia = fila["rate_incidencia"]
    estrato_incidencia = fila["Edad Incidencia"]
    try:
        # Obtiene la poblacion respectiva segun la incidencia
        estrato_a_calcular = poblaciones_estratos_calculados[estrato_incidencia]
        # Calcula los casos teoricos
        proyeccion_casos = estrato_a_calcular * rate_incidencia
        proyeccion_casos["diagnostico"] = nombre_diagnostico
        proyeccion_casos["incidencia_cada_100000"] = incidencia

    except KeyError:
        print(
            f"El problema {codigo_diagnostico} carece de un estrato de incidencia calculado. "
            f"Su estrato es: {estrato_incidencia}"
        )

El problema I080 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema I081 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema I351 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema I352 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema J860 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema J869 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema J980 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema Q231 carece de un estrato de incidencia calculado. Su estrato es: nan
El problema T820 carece de un estrato de incidencia calculado. Su estrato es: Pacientes al año con prótesis
El problema T821 carece de un estrato de incidencia calculado. Su estrato es: Pacientes al año con prótesis
El problema Z450 carece de un estrato de incidencia calculado. Su estrato es: nan


In [18]:
poblaciones_estratos_calculados

Poblacion 2017  Poblacion 2018  \
todos              Pais                             18419192        18751405   
                   Tarapacá                           342129          354940   
                   Antofagasta                        623851          645022   
                   Atacama                            304580          307835   
                   Coquimbo                           793049          807213   
...                                                      ...             ...   
mayores_a_75_anios Metropolitana de Santiago          326982          337737   
                   Los Ríos                            21284           21980   
                   Arica y Parinacota                   9993           10469   
                   Ñuble                               28994           30159   
                   SSMO                                84276           87041   

                                              Poblacion 2019  Poblacion 2020  \
todos              Pais                             19107216        19458310   
                   Tarapacá                           368906          382773   
                   Antofagasta                        668563          691854   
                   Atacama                            311307          314709   
                   Coquimbo                           821726          836096   
...                                                      ...             ...   
mayores_a_75_anios Metropolitana de Santiago          348809          360939   
                   Los Ríos                            22699           23467   
                   Arica y Parinacota                  11003           11559   
                   Ñuble                               31392           32707   
                   SSMO                                89851           92865   

                                              Poblacion 2021  Poblacion 2022  \
todos              Pais                             19678363        19828563   
                   Tarapacá                           391165          396697   
                   Antofagasta                        703746          709637   
                   Atacama                            316737          318004   
                   Coquimbo                           848079          858769   
...                                                      ...             ...   
mayores_a_75_anios Metropolitana de Santiago          374142          388666   
                   Los Ríos                            24305           25179   
                   Arica y Parinacota                  12112           12684   
                   Ñuble                               34101           35585   
                   SSMO                                96127           99705   

                                              Poblacion 2023  Poblacion 2024  \
todos              Pais                             19960889        20086377   
                   Tarapacá                           401588          406287   
                   Antofagasta                        714142          718232   
                   Atacama                            319048          319992   
                   Coquimbo                           869103          879267   
...                                                      ...             ...   
mayores_a_75_anios Metropolitana de Santiago          404171          419477   
                   Los Ríos                            26098           27076   
                   Arica y Parinacota                  13263           13834   
                   Ñuble                               37240           38995   
                   SSMO                               103512          107307   

                                              Poblacion 2025  Poblacion 2026  \
todos              Pais                             20206953        20322807   
                   Tarapacá               

In [13]:
incidencias

,Diagnostico,Incidencia (Cada 100.000),Edad Incidencia,rate_incidencia
0,C33X - Tumor maligno de la tráquea,0.260,todos,2.600000e-06
1,C340 - Tumor maligno del bronquio principal,6.900,todos,6.900000e-05
2,"C341 - Tumor maligno del lóbulo superior, bron...",6.900,todos,6.900000e-05
3,"C342 - Tumor maligno del lóbulo medio, bronqui...",6.900,todos,6.900000e-05
4,"C343 - Tumor maligno del lóbulo inferior, bron...",6.900,todos,6.900000e-05
5,C381 - Tumor maligno del mediastino anterior,1.200,todos,1.200000e-05
6,C384 - Tumor maligno de la pleura,0.035,todos,3.500000e-07
7,C450 - Mesotelioma de la pleura,0.035,todos,3.500000e-07
8,C780 - Tumor maligno secundario del pulmón,6.900,todos,6.900000e-05
9,C782 - Tumor maligno secundario de la pleura,6.900,todos,6.900000e-05
